In [1]:
#pip install pyecharts

In [25]:
import pandas as pd
import numpy as np
import pyecharts 

In [26]:
print("pyecharts version : ", pyecharts.__version__)

pyecharts version :  1.6.2


In [27]:
data = pd.read_csv('patient_data.csv', engine='python') #옵션 써야 한글 읽힘

In [28]:
data.head()

date  ID degree hospital condition sex     type  age country note
0  2020-01-19   1     1차    인천의료원        퇴원   여  중국인 관광객  30대      중국  NaN
1  2020-01-22   2     1차  국립중앙의료원        퇴원   남    우한 교민  50대      한국  NaN
2  2020-01-25   3     1차   일산명지병원        입원   남    우한 교민  50대      한국  NaN
3  2020-01-26   4     1차  분당서울대병원        퇴원   남   우한 방문자  50대      한국  NaN
4  2020-01-30   5     1차    서울의료원        입원   남   우한 방문자  30대      한국  NaN

In [29]:
data = data.fillna("")

In [30]:
data2 = data.groupby(['date', 'degree']).size()
print(data2)

date        degree
2020-01-19  1차        1
2020-01-22  1차        1
2020-01-25  1차        1
2020-01-26  1차        1
2020-01-30  1차        2
            2차        1
2020-01-31  1차        1
            2차        1
            3차        2
2020-02-01  2차        1
2020-02-02  1차        2
            3차        1
2020-02-03  1차        1
2020-02-05  1차        1
            2차        2
2020-02-06  1차        2
            2차        2
            3차        1
2020-02-09  1차        2
            2차        1
dtype: int64


In [31]:
data2 = data2.reset_index()
data2.head()

date degree  0
0  2020-01-19     1차  1
1  2020-01-22     1차  1
2  2020-01-25     1차  1
3  2020-01-26     1차  1
4  2020-01-30     1차  2

In [32]:
data2.columns = ['date', 'degree', 'count']
data2.head()

date degree  count
0  2020-01-19     1차      1
1  2020-01-22     1차      1
2  2020-01-25     1차      1
3  2020-01-26     1차      1
4  2020-01-30     1차      2

In [33]:
data2['cum_count'] = list(data2['count'].cumsum())
data2.head()

date degree  count  cum_count
0  2020-01-19     1차      1          1
1  2020-01-22     1차      1          2
2  2020-01-25     1차      1          3
3  2020-01-26     1차      1          4
4  2020-01-30     1차      2          6

In [34]:
count1 = []
for x in range(data2.shape[0]):
    if data2.loc[x, 'degree'] == "1차":
        count1.append(data2.loc[x,'count'] * 1)
    else: count1.append(data2.loc[x,'count'] * 0)
data2['cum_count1'] = list(np.cumsum(count1))

In [35]:
count2 = []
for x in range(data2.shape[0]):
    if data2.loc[x, 'degree'] == "2차":
        count2.append(data2.loc[x,'count'] * 1)
    else: count2.append(data2.loc[x,'count'] * 0)
data2['cum_count2'] = list(np.cumsum(count2))

In [36]:
count3 = []
for x in range(data2.shape[0]):
    if data2.loc[x, 'degree'] == "3차":
        count3.append(data2.loc[x,'count'] * 1)
    else: count3.append(data2.loc[x,'count'] * 0)
data2['cum_count3'] = list(np.cumsum(count3))

In [37]:
print(data2)

          date degree  count  cum_count  cum_count1  cum_count2  cum_count3
0   2020-01-19     1차      1          1           1           0           0
1   2020-01-22     1차      1          2           2           0           0
2   2020-01-25     1차      1          3           3           0           0
3   2020-01-26     1차      1          4           4           0           0
4   2020-01-30     1차      2          6           6           0           0
5   2020-01-30     2차      1          7           6           1           0
6   2020-01-31     1차      1          8           7           1           0
7   2020-01-31     2차      1          9           7           2           0
8   2020-01-31     3차      2         11           7           2           2
9   2020-02-01     2차      1         12           7           3           2
10  2020-02-02     1차      2         14           9           3           2
11  2020-02-02     3차      1         15           9           3           3
12  2020-02-

In [38]:
data_fin = data2.groupby('date').apply(max, axis=0)
data_fin = data_fin.drop(['date', 'degree', 'count'], axis=1)
print(data_fin)

            cum_count  cum_count1  cum_count2  cum_count3
date                                                     
2020-01-19          1           1           0           0
2020-01-22          2           2           0           0
2020-01-25          3           3           0           0
2020-01-26          4           4           0           0
2020-01-30          7           6           1           0
2020-01-31         11           7           2           2
2020-02-01         12           7           3           2
2020-02-02         15           9           3           3
2020-02-03         16          10           3           3
2020-02-05         19          11           5           3
2020-02-06         24          13           7           4
2020-02-09         27          15           8           4


In [39]:
# 누적확진자
import pyecharts.options as opts
from pyecharts.charts import Line, Page
from pyecharts.commons.utils import JsCode
from pyecharts.faker import Collector, Faker

In [40]:
#ver1
cum_patient = (
    Line()
    .add_xaxis(list(data_fin.index))
    .add_yaxis(
        "전체", list(data_fin['cum_count']), areastyle_opts=opts.AreaStyleOpts(opacity=1)
    )
    .add_yaxis(
        "1차", list(data_fin['cum_count1']), areastyle_opts=opts.AreaStyleOpts(opacity=0.5)
    )
    .add_yaxis(
        "2차", list(data_fin['cum_count2']), areastyle_opts=opts.AreaStyleOpts(opacity=0.5)
    )
    .add_yaxis(
        "3차", list(data_fin['cum_count3']), areastyle_opts=opts.AreaStyleOpts(opacity=0.5)
    )
    .set_colors(["maroon", "firebrick", "orangered", "coral"])
    .set_global_opts(title_opts=opts.TitleOpts(title=""),
                    xaxis_opts = opts.AxisOpts(type_ = "time", split_number=6))                   
    .set_series_opts(label_opts=opts.LabelOpts(color=["black"], font_size = 11))
)
cum_patient.render()
cum_patient.render_notebook()

In [20]:
#ver2
cum_patient2= (
    Line()
    .add_xaxis(list(data_fin.index))
    .add_yaxis(
        "", list(data_fin['cum_count']), areastyle_opts=opts.AreaStyleOpts(opacity=0.7)
    )
    .set_colors(["maroon"])
    .set_global_opts(title_opts=opts.TitleOpts(title="누적 확진자 수"),
                    xaxis_opts = opts.AxisOpts(name = "날짜", type_ = "time", split_number=6),
                    yaxis_opts = opts.AxisOpts(name = "(명)"))                   
    .set_series_opts(label_opts=opts.LabelOpts(color=["black"], font_size = 11))
)
cum_patient2.render_notebook()